In [ ]:
from src import data_handler
from src import visualization

%load_ext autoreload
%autoreload

In [ ]:
PATH_CONFIG = 'data/config/all.json'
map = data_handler.Map(PATH_CONFIG)
visualization.draw_map(map)

---

In [2]:
coords = '''
'''

coords_out = coords.replace('\n', '\n},\n')
coords_out = coords_out.replace(', ', ',\n')
coords_out = coords_out.replace('45.', '{\n\t"lat": 45.')
coords_out = coords_out.replace('18.', '\t"lon": 18.') + '\n}'

print(coords_out)

{
	"lat": 45.516213435806556,
	"lon": 18.88594918334211
},
{
	"lat": 45.51635970829011,
	"lon": 18.881399681450876
},
{
	"lat": 45.5176721724609,
	"lon": 18.872672666400817
},
{
	"lat": 45.518757696842854,
	"lon": 18.865540018341072
},
{
	"lat": 45.52445008755355,
	"lon": 18.841374677324023
},
{
	"lat": 45.52801572098923,
	"lon": 18.83329806968757
},
{
	"lat": 45.53916486597647,
	"lon": 18.79235833105591
},
{
	"lat": 45.538767671180246,
	"lon": 18.78339154939033
},
{
	"lat": 45.53965400455758,
	"lon": 18.776196779132857
},
{
	"lat": 45.54169537128007,
	"lon": 18.75789442154582
},
{
	"lat": 45.544569187322196,
	"lon": 18.746623795663638
},
{
	"lat": 45.54727919239375,
	"lon": 18.739974473015504
},
{
	"lat": 45.54931653323145,
	"lon": 18.73625110993787
},
{
	"lat": 45.55142802263625,
	"lon": 18.731034100889577
},
{
	"lat": 45.55290483711784,
	"lon": 18.721749926539008
},
{
	"lat": 45.55416646188206,
	"lon": 18.714896046769546
},
{
	"lat": 45.5546318198741,
	"lon": 18.703095975406384
},
{